In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [84]:
file_path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(file_path, 'rb').read().decode('utf-8')

In [87]:
chars=tf.strings.unicode_split(text, input_encoding='UTF-8')

In [89]:
vocab = sorted(set(text))
len(vocab)

65

In [92]:
example_input=tf.strings.unicode_split([['abc'],['xy']],input_encoding='UTF-8')
ids_from_chars=tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None)
ids=ids_from_chars(example_input)
ids

<tf.RaggedTensor [[[40, 41, 42]],
 [[63, 64]]]>

In [93]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None, invert=True)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[[b'a', b'b', b'c']],
 [[b'x', b'y']]]>

In [95]:
allids = ids_from_chars(tf.strings.unicode_split(text,input_encoding='UTF-8'))
allids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [96]:
ids_dataset = tf.data.Dataset.from_tensor_slices(allids)

In [102]:
seq_length=100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [103]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text= sequence[1:]
    return input_text, target_text

In [104]:
dataset = sequences.map(split_input_target)

In [106]:
BUFFER_SIZE=10000
BATCH_SIZE=64
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
dataset


<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [107]:
embedding_dim=256
rnn_units=1024
vocab_size=len(ids_from_chars.get_vocabulary())

In [108]:
class MyModel(tf.keras.Model):
    def __init__(self, rnn_units, embedding_dim, vocab_size):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
    def call(self, input, states=None, return_state=False, training=False):
        x= input
        x= self.embedding(x,training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x,states = self.gru(x, initial_state=states)
        x = self.dense(x)
        if return_state:
            return x, states
        else:
            return x

In [109]:
model = MyModel(rnn_units, embedding_dim, vocab_size)

In [110]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
EPOCHS=30
history=model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
141/172 [=======================>......] - ETA: 24s - loss: 1.6536

In [116]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids))

In [118]:
for example_input_batch, example_target_batch in dataset.take(1):
    example_predictions = model(example_input_batch)
    print(f'Example prediction shape : {example_predictions.shape}')

Example prediction shape : (64, 100, 66)


In [126]:
class OneStep(tf.keras.Model):
    def __init__(self, model, ids_from_chars, chars_from_ids, temperature=1):
        super().__init__(self)
        self.model = model
        self.ids_from_chars = ids_from_chars
        self.chars_from_ids = chars_from_ids
        self.temperature = temperature
        skip_ids = ids_from_chars(['[UNK]'])[:,None]
        sparse_mask = tf.SparseTensor(indices=skip_ids, 
                                      values=[-float('inf')]*len(skip_ids), 
                                      dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
    def generate_one_step(self, inputs, states=None):
        x = tf.strings.unicode_split(inputs, input_encoding='UTF-8')
        x = self.ids_from_chars(x).to_tensor()
        predicted_logits, states = self.model(x, states, return_state=True)
        predicted_logits = predicted_logits[:,-1,:]
        
        predicted_logits = predicted_logits/self.temperature
        predicted_logits = predicted_logits + self.prediction_mask
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.chars_from_ids(predicted_ids)
        return predicted_chars, states
        
        

In [127]:
onestep_model = OneStep(model, ids_from_chars, chars_from_ids)

In [132]:
next_char=tf.constant(['ROMEO:'])
result = [next_char]

for i in range(1000):
    next_char, states = onestep_model.generate_one_step(next_char, states)
    result.append(next_char)

result = tf.strings.join(result)


In [134]:
result[0].numpy()

b"ROMEO:\nWhou fear mut is a campol' frey Wimour so fealatly.\n\nPirst:\nAs he't moned with Plivized\nOt, the charty and my contentings, go'tlys that\nTho caul on the dight wer receit.\n\nTRING LEDWARD II VI:\nWhat, his redour eyeps a fail\nTreakes pord to die duedion mant.\n\nCAMINCEL:\nVo thy feelly twance, thoued re put inseambed my tach\nHot eorp. usen:\nI say, out egamenspeds?\nWhere thou ore gone; lets what, incliar in mo\nnos their them woble I marding mour\nThew's nebsey, and outher alf than gave upon the merney?\nSopelvey in th sheefs reasen's it oners and all priad you,\nMary strept she dour frawer stire baghtle\nWhe came and betten have sons; it at whint\nWhich sut ut bood that it youre pase: and thee agon the mort.\n\nCLIOND:\nThe fats, duth doobs of consperily and evernds this?\n\npordoRY:\nA prismio, that are it you a te:\nYour love, sidst shrugh\nWhich havo mad menm'd he dsand arate break down\nHore.\nTo som.\n\nRICHARD:\nSiren my foll3 AVles conwilats,--\n\nFhold Prover